# MolMap ensemble (dual-path approach)

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]='0'

import tensorflow as tf

physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

### Load data

In [ ]:
'Set the file name of your data'
data_name = "CYP450.csv.gz"

In [2]:
import numpy as np
import pandas as pd

data = pd.read_csv("../../../data/" + data_name, compression='gzip')

Set up (adapt according to your data)

In [ ]:
# keep desired columns
data = data[['smiles', 'label_2c9']]

# drop molecules with NaN activity
data = data.dropna(subset = ["label_2c9"])

# set SMILES
smi = data['smiles'].tolist()

# set Y
Y = pd.get_dummies(data['label_2c9']).values

# number of active and inactive molecules
print("Inactive (0):", Y[:,1].tolist().count(0))
print("Active (1):", Y[:,1].tolist().count(1))

### MolMap Ensemble

In [ ]:
'Set size of class 0 (minority class) for each model'
size_0 = 500

partitions = int(np.floor(Y[:,1].tolist().count(0)/size_0))
print("Total number of models:", partitions)

In [6]:
'Get classes 0 and 1 separately'
class_0 = data[data["label_2c9"] == 0.0]
class_1 = data[data["label_2c9"] == 1.0]

# shuffle class 0 before dividing it into parts
class_0 = class_0.sample(frac=1).reset_index(drop=True)

In [7]:
from molmap import MolMap
from molmap import feature
from model import MultiClassEstimator
import sys
sys.path.append("../../../src")
from utils import Rdsplit

aucs = []
# for each model:
for i in range (0, partitions):
    
    # subset of data for model i
    subdata = pd.concat([class_0[i*size_0:(i+1)*size_0], class_1])
    
    # get smiles
    smi = subdata['Smiles']
    
    # set Y
    Y = pd.get_dummies(subdata['% inhibition']).values
    
    # compute MolDs
    mp1 = MolMap(ftype='descriptor', metric='cosine',)
    mp1.fit(verbose=0, method='umap', min_dist=0.1, n_neighbors=15)
    
    # compute FFs
    bitsinfo = feature.fingerprint.Extraction().bitsinfo
    flist = bitsinfo[bitsinfo.Subtypes.isin(['PubChemFP', 'MACCSFP', 'PharmacoErGFP'])].IDs.tolist()
    mp2 = MolMap(ftype = 'fingerprint', fmap_type = 'scatter', flist = flist) 
    mp2.fit(method = 'umap',  min_dist = 0.1, n_neighbors = 15, verbose = 0)
    
    # get Fmaps
    X1 = mp1.batch_transform(smi)
    X2 = mp2.batch_transform(smi)
    
    # split train, val, test
    train_idx, valid_idx, test_idx = Rdsplit(subdata, random_state = 888)
    trainX = (X1[train_idx], X2[train_idx])
    validX = (X1[valid_idx], X2[valid_idx])
    testX = (X1[test_idx], X2[test_idx])
    trainY = Y[train_idx]
    validY = Y[valid_idx]
    testY = Y[test_idx]
    
    # model 
    clf = MultiClassEstimator(n_outputs=trainY.shape[1], 
                              fmap_shape1 = X1.shape[1:],
                              fmap_shape2 = X2.shape[1:],
                              metric='ROC', 
                              dense_layers = [128, 64],  gpuid = 0, epochs = 100,
                              ) 
    # fit model
    clf.fit(trainX, trainY, validX, validY)
    
    # score 
    auc = clf.score(testX, testY) 
    aucs.append(auc)

2022-06-20 11:23:52,251 - INFO - [bidd-molmap] - Applying grid feature map(assignment), this may take several minutes(1~30 min)
2022-06-20 11:23:54,303 - INFO - [bidd-molmap] - Finished
2022-06-20 11:23:59,897 - INFO - [bidd-molmap] - Applying naive scatter feature map...
2022-06-20 11:23:59,909 - INFO - [bidd-molmap] - Finished


100%|##########| 12169/12169 [05:54<00:00, 34.28it/s]


9737 1216 1216
MultiClassEstimator(epochs=100, fmap_shape1=(37, 37, 13),
                    fmap_shape2=(72, 72, 3), gpuid='0', n_outputs=2)
epoch: 0001, loss: 0.2825 - val_loss: 0.2440; auc: 0.7762 - val_auc: 0.7742                                                                                                    
epoch: 0002, loss: 0.2453 - val_loss: 0.2258; auc: 0.8434 - val_auc: 0.8537                                                                                                    
epoch: 0003, loss: 0.2218 - val_loss: 0.1935; auc: 0.8624 - val_auc: 0.8779                                                                                                    
epoch: 0004, loss: 0.1993 - val_loss: 0.1885; auc: 0.8749 - val_auc: 0.8913                                                                                                    
epoch: 0005, loss: 0.1875 - val_loss: 0.1837; auc: 0.8839 - val_auc: 0.8977                                                                               

100%|##########| 12169/12169 [06:50<00:00, 29.64it/s]


9737 1216 1216
MultiClassEstimator(epochs=100, fmap_shape1=(37, 37, 13),
                    fmap_shape2=(72, 72, 3), gpuid='0', n_outputs=2)
epoch: 0001, loss: 0.2831 - val_loss: 0.2455; auc: 0.7591 - val_auc: 0.7487                                                                                                    
epoch: 0002, loss: 0.2459 - val_loss: 0.2269; auc: 0.8462 - val_auc: 0.8298                                                                                                    
epoch: 0003, loss: 0.2193 - val_loss: 0.1981; auc: 0.8687 - val_auc: 0.8448                                                                                                    
epoch: 0004, loss: 0.1964 - val_loss: 0.1880; auc: 0.8798 - val_auc: 0.8522                                                                                                    
epoch: 0005, loss: 0.1857 - val_loss: 0.1887; auc: 0.8905 - val_auc: 0.8574                                                                               

...

9737 1216 1216
MultiClassEstimator(epochs=100, fmap_shape1=(37, 37, 13),
                    fmap_shape2=(72, 72, 3), gpuid='0', n_outputs=2)
epoch: 0001, loss: 0.2831 - val_loss: 0.2446; auc: 0.7642 - val_auc: 0.7666                                                                                                    
epoch: 0002, loss: 0.2462 - val_loss: 0.2267; auc: 0.8448 - val_auc: 0.8347                                                                                                    
epoch: 0003, loss: 0.2220 - val_loss: 0.1981; auc: 0.8685 - val_auc: 0.8521                                                                                                    
epoch: 0004, loss: 0.1956 - val_loss: 0.1853; auc: 0.8815 - val_auc: 0.8585                                                                                                    
epoch: 0005, loss: 0.1842 - val_loss: 0.1993; auc: 0.8926 - val_auc: 0.8649                                                                               

100%|##########| 12169/12169 [06:05<00:00, 33.32it/s]


9737 1216 1216
MultiClassEstimator(epochs=100, fmap_shape1=(37, 37, 13),
                    fmap_shape2=(72, 72, 3), gpuid='0', n_outputs=2)
epoch: 0001, loss: 0.2828 - val_loss: 0.2454; auc: 0.7584 - val_auc: 0.7475                                                                                                    
epoch: 0002, loss: 0.2465 - val_loss: 0.2282; auc: 0.8423 - val_auc: 0.8253                                                                                                    
epoch: 0003, loss: 0.2215 - val_loss: 0.1985; auc: 0.8660 - val_auc: 0.8428                                                                                                    
epoch: 0004, loss: 0.1963 - val_loss: 0.1915; auc: 0.8792 - val_auc: 0.8521                                                                                                    
epoch: 0005, loss: 0.1855 - val_loss: 0.1821; auc: 0.8907 - val_auc: 0.8580                                                                               

100%|##########| 12169/12169 [06:04<00:00, 33.40it/s]


9737 1216 1216
MultiClassEstimator(epochs=100, fmap_shape1=(37, 37, 13),
                    fmap_shape2=(72, 72, 3), gpuid='0', n_outputs=2)
epoch: 0001, loss: 0.2829 - val_loss: 0.2451; auc: 0.7543 - val_auc: 0.7492                                                                                                    
epoch: 0002, loss: 0.2463 - val_loss: 0.2272; auc: 0.8412 - val_auc: 0.8256                                                                                                    
epoch: 0003, loss: 0.2218 - val_loss: 0.2000; auc: 0.8665 - val_auc: 0.8442                                                                                                    
epoch: 0004, loss: 0.1970 - val_loss: 0.1876; auc: 0.8784 - val_auc: 0.8517                                                                                                    
epoch: 0005, loss: 0.1865 - val_loss: 0.1867; auc: 0.8891 - val_auc: 0.8576                                                                               

In [9]:
print("Total number of models trained:", partitions)
print("Mean of accuracies:", round(np.mean(aucs), 3))
print("Variance of accuracies:", round(np.var(aucs), 3))

[0.9474586224667122,
 0.904876616450341,
 0.8953007586380469,
 0.9028032454847255,
 0.9071896834813457,
 0.8932753268855091,
 0.9126187993624084,
 0.9032227135991564,
 0.8991718500940806,
 0.9047807380241853,
 0.9124629969199055,
 0.8966909958173036,
 0.8888169800692721,
 0.9029590479272283,
 0.8992677285202364,
 0.9028272150912644,
 0.9097184769712003,
 0.9056196742530471,
 0.9072376226944234,
 0.8999268927000563]